# <center> Enviar datos a FuSA </center>

Este notebook apunta a documentar cómo conectarse a la API de FuSA para subir datos.

Contacto: Matthieu Vernier (mvernier@inf.uach.cl), Victor Várgas (victorvargassandoval93@gmail.com)

# 1. Datos para conectarse a la API de FuSA

In [1]:
#La URL del servidor donde está alojado la API. Este dato es susceptible de cambiar
FUSA_SERVER="http://45.79.170.31:8000" #URL del servidor de desarrollo

#Su clave única para conectarse a la API
API_KEY="zIkYk2VwBiC762O4yFSd" #Clave de demostración para el servidor de desarrollo.
HEADERS = { "X-Api-Key": API_KEY }

#Endpoints. Ver documentación: http://45.79.170.31:8000/docs
ENDPOINT="audios"

#URI de la API FuSA
URI = f"{FUSA_SERVER}/{ENDPOINT}"

# 2. Seleccionar un archivo audio que enviar a la API de FuSA

In [2]:
#Elegir la ruta del archivo que subir
FILEPATH="data/cow.wav"

In [3]:
#Verificar que el archivo existe
import os
if os.path.exists(FILEPATH):
    print("El archivo existe.")
else:
    print("El archivo no existe.")

El archivo existe.


In [4]:
#Escuchar el archivo
import IPython.display as ipd
ipd.Audio(FILEPATH)

# 3. Calcular el tamaño y la duración del archivo audio

In [5]:
import os
SIZE = os.path.getsize(FILEPATH)
print(SIZE)

514727


In [6]:
#DURATION
import librosa #requiere instalar con "pip install librosa"
DURATION=librosa.get_duration(filename=FILEPATH)
print(DURATION)

3.5622708333333333


# 4. Convertir el dato audio en base64

In [7]:
import base64

byte_content = open(FILEPATH, "rb").read()
base64_bytes = base64.b64encode(byte_content)
base64_string = base64_bytes.decode("utf-8")

DATA = base64_string

# 3. Ingresar los metadatos describiendo el archivo

- El archivo:

In [8]:
NAME="Una vaca en Valdivia" #un nombre para describir el archivo
FORMAT="wav" #wav, mp3, etc.
DESCRIPTION="una descripción"
RECORDING_DEVICE="smartphone" #["smartphone","sonometer"]

- La ubicación de la grabación:

In [9]:
LATITUDE=-39.81422
LONGITUDE=-73.24589

- La fecha de grabación y de upload:

In [10]:
RECORDED_AT = "20/01/2020 09:50" #Conservar el formato "%d/%m/%Y %H:%M"

In [11]:
import time
import datetime

RECORDED_AT = time.mktime(datetime.datetime.strptime(RECORDED_AT,"%d/%m/%Y %H:%M").timetuple())

In [12]:
from datetime import datetime

# current date and time
now = datetime.now()

timestamp = datetime.timestamp(now)

UPLOADED_AT = timestamp

- El usuario:

In [13]:
CATEGORY="citizen" #"citizen" o "sensor"
USERNAME="mvernier" #deberia ser un nombre único para identificar el usuario

- Las etiquetas del audio: 

In [15]:
SOURCE_TAGS=["sirena","alarma"]

In [16]:
import json

CONTENT={
  "name": NAME,
  "description": DESCRIPTION,
  "format": FORMAT,
  "size": SIZE,
  "duration": DURATION,
  "recorded_at": RECORDED_AT,
  "uploaded_at": UPLOADED_AT,
  "latitude": LATITUDE,
  "longitude": LONGITUDE,
  "data": DATA,
  "recording_device":RECORDING_DEVICE,
  "user": {
    "category": CATEGORY,
    "username": USERNAME
  },
  "tags": [
    {
      "username": USERNAME,
      "source_tags": SOURCE_TAGS
    }
  ]
}

CONTENT=json.dumps(CONTENT)

In [17]:
print(CONTENT)

{"name": "Una vaca en Valdivia", "description": "una descripci\u00f3n", "format": "wav", "size": 514727, "duration": 3.5622708333333333, "recorded_at": 1579510200.0, "uploaded_at": 1630595936.233876, "latitude": -39.81422, "longitude": -73.24589, "data": "...", "recording_device": "smartphone", "user": {"category": "citizen", "username": "mvernier"}, "tags": [{"username": "mvernier", "source_tags": ["animal", "vaca"]}]}


# 4. Enviar un audio a la API FuSA (POST)

In [18]:
import requests

response = requests.post(URI, data=CONTENT, headers=HEADERS)

print("Código de su consulta: "+str(response.status_code))
print(response.content)

Código de su consulta: 200
b'{"data":{"name":"Una vaca en Valdivia","description":"una descripci\xc3\xb3n","format":"wav","size":514727.0,"duration":3.5622708333333333,"recorded_at":1579510200,"uploaded_at":1630595936,"latitude":-39.81422,"longitude":-73.24589,"data":"...","recording_device":"smartphone","user":{"category":"citizen","username":"mvernier"},"tags":[{"username":"mvernier","source_tags":["animal","vaca"]}],"chunks":null},"code":200,"message":"Audio added successfully."}'
